In [24]:
print(all_players)

NameError: name 'all_players' is not defined

#MILB DATA FROM TABLE DOWNLOADS

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import glob
import os
def download_fangraphs_data(data_type='bat', data_level='standard', download_dir='default_path'):
    # Define the URLs for different data types and levels
    urls = {
        'bat_standard': 'https://www.fangraphs.com/leaders/minor-league?pos=all&level=0&lg=all&stats=bat&qual=10&type=0&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false&players=&sort=18,1&pageitems=2000000000&pg=0',
        'bat_advanced': 'https://www.fangraphs.com/leaders/minor-league?pos=all&level=0&lg=all&stats=bat&qual=10&type=1&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false&players=&sort=18,1&pageitems=2000000000&pg=0',
        'pit_standard': 'https://www.fangraphs.com/leaders/minor-league?pos=all&level=0&lg=all&stats=pit&qual=10&type=0&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false&players=&sort=18,1&pageitems=2000000000&pg=0',
        'pit_advanced': 'https://www.fangraphs.com/leaders/minor-league?pos=all&level=0&lg=all&stats=pit&qual=10&type=1&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false&players=&sort=18,1&pageitems=2000000000&pg=0',
        'bat_batted': 'https://www.fangraphs.com/leaders/minor-league?pos=all&level=0&lg=all&stats=bat&qual=10&type=2&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false&players=&sort=18,1&pageitems=2000000000&pg=0',
        'bat_batted': 'https://www.fangraphs.com/leaders/minor-league?pos=all&level=0&lg=all&stats=bat&qual=10&type=2&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false&players=&sort=18,1&pageitems=2000000000&pg=0',
        'pit_batted': 'https://www.fangraphs.com/leaders/minor-league?pos=all&level=0&lg=all&stats=pit&qual=10&type=2&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false&players=&sort=18,1&pageitems=2000000000&pg=0',       
    }

    # Set the download directory
    if download_dir == 'default_path':
        download_dir = r"C:\Users\vile3\Documents\GitHub\baseball_pred\SCRAPE_FANGRAPHS"

    # Initialize the Options object
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-popup-blocking")

    # Set download preferences
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })

    # Initialize the Chrome driver with the options
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    # Construct the key for the URL dictionary
    key = f'{data_type}_{data_level}'
    url = urls.get(key, None)
    if not url:
        print(f"Invalid data type or level: {data_type}, {data_level}")
        return

    # Open the webpage
    driver.get(url)

    # Wait for the 'Export Data' button to be clickable
    wait = WebDriverWait(driver, 30)
    export_button = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'Export Data')))
    # Click the 'Export Data' button
    export_button.click()

    # Wait for the download to start
    wait = WebDriverWait(driver, 5)

    # Close the browser when done
    driver.quit()

# Example usage:
data_types_levels = [
    ('bat', 'standard'),
    ('bat', 'advanced'),
    ('bat', 'batted'),
    ('pit', 'standard'),
    ('pit', 'advanced'),
    ('pit', 'batted')
]


# from concurrent.futures import ThreadPoolExecutor
    # Rename the downloaded csv files to match their data_type and data_level
for  data_type, data_level in data_types_levels:
    download_dir = f"C:\\Users\\vile3\\Documents\\GitHub\\baseball_pred\\SCRAPE_FANGRAPHS\\{data_type.capitalize()}\\{data_level.capitalize()}"
    download_fangraphs_data(data_type=data_type, data_level=data_level, download_dir=download_dir)
    os.rename(f"{download_dir}/fangraphs-minor-league-leaders.csv", f"{download_dir}/{data_type}_{data_level}.csv")
    

# data_types_levels = [
#     ('bat', 'standard'),
#     ('bat', 'advanced'),
#     ('bat', 'batted'),
#     ('pit', 'standard'),
#     ('pit', 'advanced'),
#     ('pit', 'batted')
# ]

# def download_wrapper(args):
#     data_type, data_level = args
#     download_dir = f"C:\\Users\\vile3\\Documents\\GitHub\\baseball_pred\\SCRAPE_FANGRAPHS\\{data_type.capitalize()}\\{data_level.capitalize()}"
#     download_fangraphs_data(data_type=data_type, data_level=data_level, download_dir=download_dir)

# # Use ThreadPoolExecutor to run each download concurrently
# with ThreadPoolExecutor(max_workers=len(data_types_levels)) as executor:
#     executor.map(download_wrapper, data_types_levels)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\vile3\\Documents\\GitHub\\baseball_pred\\SCRAPE_FANGRAPHS\\Pit\\Batted\\fangraphs-minor-league-leaders.csv' -> 'C:\\Users\\vile3\\Documents\\GitHub\\baseball_pred\\SCRAPE_FANGRAPHS\\Pit\\Batted/pit_batted.csv'

In [ ]:
print(download_dir)

C:\Users\vile3\Documents\GitHub\baseball_pred\SCRAPE_FANGRAPHS\Pit\Batted


In [2]:
import requests
import os
import json
import csv
from bs4 import BeautifulSoup
import pandas as pd
import glob

# Function to clean HTML tags from data
def clean_html(raw_html):
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text()

# Function to download data as JSON
def download_api_data_json(url, download_dir, file_name):
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        if 'application/json' in response.headers.get('Content-Type'):
            os.makedirs(download_dir, exist_ok=True)
            file_path = os.path.join(download_dir, f'{file_name}.json')
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(response.text)
            return file_path
        else:
            print("The response content is not in JSON format.")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to convert JSON to CSV and clean HTML
import json
import csv
from bs4 import BeautifulSoup

# Function to clean HTML tags from data
def clean_html(raw_html):
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text()

def convert_json_to_csv(json_file_path, csv_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        # Parse the JSON data into a Python object
        json_data = json.load(file)

    # Access the list of data under the 'data' key
    data = json_data.get('data', [])

    # Check if data is a list and proceed
    if isinstance(data, list) and data:
        # Clean the HTML content in the JSON data
        for entry in data:
            for key, value in entry.items():
                if isinstance(value, str) and ('<' in value and '>' in value):
                    entry[key] = clean_html(value)

        # Determine the fieldnames from the keys of the first record in your JSON data
        fieldnames = data[0].keys()

        # Write to CSV
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            for entry in data:
                writer.writerow(entry)
    else:
        print("JSON data is not a list or is empty.")





# Function to merge CSV files
def merge_csv_files(directory_path, unique_identifier):
    # Create a pattern for all CSV files in the directory
    csv_pattern = os.path.join(directory_path, '*.csv')
    csv_files = glob.glob(csv_pattern)
    # Initialize an empty DataFrame for the combined data
    combined_df = pd.DataFrame()

    for file in csv_files:
        # Read the current CSV into a DataFrame
        df = pd.read_csv(file)

        if combined_df.empty:
            # If the combined DataFrame is empty, start with the first file's data
            combined_df = df
        else:
            # Merge with an outer join to add new columns (stats) and rows (players)
            combined_df = pd.merge(combined_df, df, on=unique_identifier, how='outer', suffixes=('', '_dup'))

            # Handle duplicate columns if they exist
            for column in combined_df.columns:
                if '_dup' in column:
                    original_column = column.replace('_dup', '')
                    # If the original column is not NaN, keep it, otherwise use the duplicate column's value
                    combined_df[original_column] = combined_df.apply(
                        lambda row: row[original_column] if not pd.isnull(row[original_column]) else row[column],
                        axis=1
                    )
                    # Drop the duplicate column
                    combined_df.drop(columns=[column], inplace=True)

    # Save the combined DataFrame to a new CSV file
    combined_csv_path = os.path.join(directory_path, 'combined_data.csv')
    combined_df.to_csv(combined_csv_path, index=False)
    print(f"Combined CSV saved at: {combined_csv_path}")

# Define the API URLs
api_urls = {
    'bat_standard': 'https://www.fangraphs.com/api/leaders/minor-league/data?pos=all&level=0&lg=all&stats=bat&qual=10&type=0&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false',
    'bat_advanced': 'https://www.fangraphs.com/api/leaders/minor-league/data?pos=all&level=0&lg=all&stats=bat&qual=10&type=1&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false',
    'bat_batted': 'https://www.fangraphs.com/api/leaders/minor-league/data?pos=all&level=0&lg=all&stats=bat&qual=10&type=2&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false',
    'pit_standard': 'https://www.fangraphs.com/api/leaders/minor-league/data?pos=all&level=0&lg=all&stats=pit&qual=10&type=0&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false',
    'pit_advanced': 'https://www.fangraphs.com/api/leaders/minor-league/data?pos=all&level=0&lg=all&stats=pit&qual=10&type=1&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false',
    'pit_batted': 'https://www.fangraphs.com/api/leaders/minor-league/data?pos=all&level=0&lg=all&stats=pit&qual=10&type=2&team=&season=2006&seasonEnd=2023&org=all&ind=0&splitTeam=false',
    'mlbbat_standard': 'https://www.fangraphs.com/api/leaders/major-league/data?age=&pos=all&stats=bat&lg=all&qual=10&season=2023&season1=2006&startdate=2023-03-01&enddate=2023-11-01&month=0&hand=&team=0&pageitems=2000000&pagenum=1&ind=0&rost=0&players=&type=0&postseason=&sortdir=default&sortstat=AVG',
    'mlbbat_advanced': 'https://www.fangraphs.com/api/leaders/major-league/data?age=&pos=all&stats=bat&lg=all&qual=10&season=2023&season1=2006&startdate=2023-03-01&enddate=2023-11-01&month=0&hand=&team=0&pageitems=2000000&pagenum=1&ind=0&rost=0&players=&type=1&postseason=&sortdir=default&sortstat=wRC%2B',
    'mlbbat_batted': 'https://www.fangraphs.com/api/leaders/major-league/data?age=&pos=all&stats=bat&lg=all&qual=10&season=2023&season1=2006&startdate=2023-03-01&enddate=2023-11-01&month=0&hand=&team=0&pageitems=2000000&pagenum=1&ind=0&rost=0&players=&type=2&postseason=&sortdir=default&sortstat=Hard%25',    'mlbpit_standard': 'https://www.fangraphs.com/api/leaders/major-league/data?pos=all&stats=pit&lg=all&qual=10&type=0&season=2006&seasonEnd=2023&ind=0',
    'mlbpit_standard': 'https://www.fangraphs.com/api/leaders/major-league/data?age=&pos=all&stats=pit&lg=all&qual=10&season=2023&season1=2006&startdate=2023-03-01&enddate=2023-11-01&month=0&hand=&team=0&pageitems=2000000&pagenum=1&ind=0&rost=0&players=&type=0&postseason=&sortdir=default&sortstat=SO',
    'mlbpit_advanced': 'https://www.fangraphs.com/api/leaders/major-league/data?age=&pos=all&stats=pit&lg=all&qual=10&season=2023&season1=2006&startdate=2023-03-01&enddate=2023-11-01&month=0&hand=&team=0&pageitems=2000000&pagenum=1&ind=0&rost=0&players=&type=1&postseason=&sortdir=default&sortstat=SIERA',
    'mlbpit_batted': 'https://www.fangraphs.com/api/leaders/major-league/data?age=&pos=all&stats=pit&lg=all&qual=10&season=2023&season1=2006&startdate=2023-03-01&enddate=2023-11-01&month=0&hand=&team=0&pageitems=2000000&pagenum=1&ind=0&rost=0&players=&type=2&postseason=&sortdir=default&sortstat=Hard%25',
}
# Function to process selected data types
def process_selected_data_types(selected_data_types):
    for key in selected_data_types:
        url = api_urls.get(key)
        if url:
            data_type, data_level = key.split('_')
            download_dir = os.path.join(r'C:\Users\vile3\Documents\GitHub\baseball_pred\SCRAPE_FANGRAPHS', 'API' + data_type.capitalize())
            file_name = f'{data_type}_{data_level}'
            
            # Download the data as JSON
            json_file_path = download_api_data_json(url, download_dir, file_name)
            
            if json_file_path:
                # Convert JSON to CSV
                csv_file_path = os.path.join(download_dir, f'{file_name}.csv')
                convert_json_to_csv(json_file_path, csv_file_path)
                print(f"Downloaded and converted {file_name} to CSV.")
        else:
            print(f"No URL found for {key}")

# Usage example for a single data type

# process_selected_data_types(['bat_standard'])
process_selected_data_types(['mlbbat_standard'])
process_selected_data_types(['mlbbat_advanced'])
process_selected_data_types(['mlbbat_batted'])
process_selected_data_types(['mlbpit_standard'])
process_selected_data_types(['mlbpit_advanced'])
process_selected_data_types(['mlbpit_batted'])
# Usage example for all data types
# process_selected_data_types(api_urls.keys())

# After processing the data types, you can merge the CSV files as before:
bat_directory = r'C:\Users\vile3\Documents\GitHub\baseball_pred\SCRAPE_FANGRAPHS\APIMlbbat'
pit_directory = r'C:\Users\vile3\Documents\GitHub\baseball_pred\SCRAPE_FANGRAPHS\APIMlbpit'

merge_csv_files(bat_directory, 'playerid')
merge_csv_files(pit_directory, 'playerid')

Downloaded and converted mlbbat_standard to CSV.
Downloaded and converted mlbbat_advanced to CSV.
Downloaded and converted mlbbat_batted to CSV.
Downloaded and converted mlbpit_standard to CSV.
Downloaded and converted mlbpit_advanced to CSV.
Downloaded and converted mlbpit_batted to CSV.
Combined CSV saved at: C:\Users\vile3\Documents\GitHub\baseball_pred\SCRAPE_FANGRAPHS\APIMlbbat\combined_data.csv
Combined CSV saved at: C:\Users\vile3\Documents\GitHub\baseball_pred\SCRAPE_FANGRAPHS\APIMlbpit\combined_data.csv
